# debug weird pvals

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

import numpy as np
import pandas as pd
import plotly.express as px
import upath

import extras

loading extras module


In [3]:
path_root = upath.UPath("gs://liulab/differential_composition_and_expression/20230317_06h00m45s")

In [5]:
subset_gene_stats = pd.read_parquet(
    path_root
    / "experiment_id=264/malignant_means=0.72,0.7/log2_fc=1.00/run_id=04/deg_analysis/gene_stats_malignant_cibersortx.parquet"
)

print(subset_gene_stats.shape)

# subset_gene_stats[columns].sort_values(by="-log10_pval_adjusted_bh", ascending=False)
print(subset_gene_stats["significant_bh_fdr=0.10"].value_counts())

fig = px.line(
    subset_gene_stats.sort_values("pval").reset_index(),
    x="gene_symbol",
    y="pval",
)
fig

(16063, 10)
False    16036
True        27
Name: significant_bh_fdr=0.10, dtype: int64


## are these being overwritten by multipletests?

In [13]:
experiment_path = path_root / "experiment_id=264/malignant_means=0.72,0.7/log2_fc=1.00/run_id=04"
pattern = experiment_path / "cibersortx" / "**" / "*Malignant*txt"
# print files matching pattern
for path in experiment_path.glob("cibersortx/**/*Malignant*txt"):
    print(path)

gs://liulab/differential_composition_and_expression/20230317_06h00m45s/experiment_id=264/malignant_means=0.72,0.7/log2_fc=1.00/run_id=04/cibersortx/outdir/CIBERSORTxHiRes_NA_Malignant_Window36.txt


In [14]:
import dask.dataframe as dd

In [15]:
from run_everything import compute_stats

In [30]:
rnaseq_malignant_cibersortx = (
    dd.read_csv(pattern, sep="\t")
    .rename(columns={"GeneSymbol": "gene_symbol"})
    .set_index("gene_symbol")
    .compute()
)

rnaseq_malignant_cibersortx

In [60]:
import helpers
import scipy.stats


def compute_pval_per_gene(rnaseq: pd.DataFrame) -> pd.DataFrame:
    rnaseq = rnaseq.copy()
    rnaseq.columns = pd.MultiIndex.from_tuples(
        rnaseq.columns.str.split("/", expand=True),
        names=["group_id", "sample_id"],
    )
    rnaseq = rnaseq.stack(["group_id", "sample_id"])
    rnaseq_groupby = rnaseq.groupby("gene_symbol", group_keys=True)
    # print group names
    print(rnaseq_groupby.groups.keys())
    # print size of each group
    print(rnaseq_groupby.size())
    # get the group for key "CUX13"
    gene_rnaseq = rnaseq_groupby.get_group("PAPD4")

    def compute_pval_for_gene(gene_rnaseq: pd.Series):
        group_a = gene_rnaseq.xs("a", level="group_id")
        group_b = gene_rnaseq.xs("b", level="group_id")
        statistic, pvalue = scipy.stats.mannwhitneyu(group_a, group_b)
        # print(statistic, pvalue)
        return pd.DataFrame({"pval": [pvalue]})

    gene_stats = rnaseq_groupby.apply(compute_pval_for_gene)
    # gene_stats = helpers.deg_analysis.compute_stats(rnaseq_groupby, "group_id", "a", "b")
    # scipy.stats.mannwhitneyu()
    return gene_stats


gene_stats = compute_pval_per_gene(rnaseq_malignant_cibersortx)
print(gene_stats)

dict_keys(['A1BG', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAT', 'AAGAB', 'AAK1', 'AAMP', 'AANAT', 'AARS', 'AARS2', 'AARSD1', 'AASDH', 'AASDHPPT', 'AASS', 'AATF', 'AATK', 'ABAT', 'ABCA1', 'ABCA10', 'ABCA11P', 'ABCA12', 'ABCA13', 'ABCA17P', 'ABCA2', 'ABCA3', 'ABCA4', 'ABCA5', 'ABCA6', 'ABCA7', 'ABCA8', 'ABCA9', 'ABCB1', 'ABCB10', 'ABCB11', 'ABCB4', 'ABCB5', 'ABCB6', 'ABCB7', 'ABCB8', 'ABCB9', 'ABCC1', 'ABCC10', 'ABCC11', 'ABCC2', 'ABCC3', 'ABCC4', 'ABCC5', 'ABCC6', 'ABCC6P2', 'ABCC9', 'ABCD1', 'ABCD2', 'ABCD3', 'ABCD4', 'ABCE1', 'ABCF1', 'ABCF2', 'ABCF3', 'ABCG1', 'ABCG2', 'ABCG4', 'ABCG5', 'ABHD1', 'ABHD10', 'ABHD11', 'ABHD12', 'ABHD12B', 'ABHD13', 'ABHD14A', 'ABHD14B', 'ABHD15', 'ABHD2', 'ABHD3', 'ABHD4', 'ABHD5', 'ABHD6', 'ABHD8', 'ABI1', 'ABI2', 'ABI3', 'ABI3BP', 'ABL1', 'ABL2', 'ABLIM1', 'ABLIM2', 'ABLIM3', 'ABO', 'ABP1', 'ABR', 'ABT1', 'ABTB1', 'ABTB2', 'ACAA1', 'ACAA2', 'ACACA', 'ACACB', 'ACAD10', 'ACAD11', 'ACAD8', 'ACAD9', 'ACADL', 'ACADM', 'ACADS', 'ACADSB', 'ACA

In [20]:
subset_gene_stats = compute_stats(rnaseq_malignant_cibersortx, "a", "b")

hi from /home/jupyter/deconv/helpers/deg_analysis.py


In [54]:
gene_stats.describe()

count    1.606300e+04
mean     5.925708e-01
std      3.257931e-01
min      3.257017e-13
25%      3.027162e-01
50%      6.155779e-01
75%      9.237363e-01
max      1.000000e+00
dtype: float64

In [55]:
fig = px.line(
    gene_stats.sort_values("pval").reset_index(),
    x="gene_symbol",
    y="pval",
)
fig

/tmp/ipykernel_24884/2425076785.py:2: FutureWarning:

In a future version of pandas all arguments of Series.sort_values will be keyword-only.



ValueError: No axis named pval for object type Series